In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from adam_prepare import titanic_pipeline

Let's read in the data using the titanic pipeline function!

In [3]:
train, val, test = titanic_pipeline()
train.head()

,survived,age,sibsp,parch,fare,alone,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,0,19.0,1,0,53.1000,0,1,1,0,0,0,0,1
45,0,29.0,0,0,8.0500,1,1,0,0,1,0,0,1
28,1,29.0,0,0,7.8792,1,0,0,0,1,0,1,0
633,0,29.0,0,0,0.0000,1,1,1,0,0,0,0,1
403,0,28.0,1,0,15.8500,0,1,0,0,1,0,0,1


Let's define a function to create X and y splits of our data.

In [4]:
def xy_split(df):
    
    return df.drop(columns = ['survived']), df.survived

In [5]:
X_train, y_train = xy_split(train)
X_val, y_val = xy_split(val)

X_train.head()

,age,sibsp,parch,fare,alone,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,19.0,1,0,53.1000,0,1,1,0,0,0,0,1
45,29.0,0,0,8.0500,1,1,0,0,1,0,0,1
28,29.0,0,0,7.8792,1,0,0,0,1,0,1,0
633,29.0,0,0,0.0000,1,1,1,0,0,0,0,1
403,28.0,1,0,15.8500,0,1,0,0,1,0,0,1


Now it's time to build our [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model!

In [6]:
seed = 42

logreg = LogisticRegression(random_state = seed)

logreg.fit(X_train, y_train)

logreg.score(X_train, y_train), logreg.score(X_val, y_val)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8025682182985554, 0.8507462686567164)

In [7]:
logreg.coef_

array([[-0.03854834, -0.48079229, -0.17174881,  0.00312492, -0.5621861 ,
        -2.44550303,  1.45982081,  0.5902067 , -0.46713545,  0.43766642,
         0.80473749,  0.16931327]])

In [8]:
pd.DataFrame({'feature': X_train.columns,
              'coefficient': logreg.coef_[0]})

,feature,coefficient
0,age,-0.038548
1,sibsp,-0.480792
2,parch,-0.171749
3,fare,0.003125
4,alone,-0.562186
5,sex_male,-2.445503
6,class_First,1.459821
7,class_Second,0.590207
8,class_Third,-0.467135
9,embark_town_Cherbourg,0.437666
